<a href="https://colab.research.google.com/github/PiyushChall/Handwritten_Digit_Recognizer/blob/main/HandwrittenDigitRecognizer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Handwritten Digit Recognition with Neural Networks**





# Some Installations

In [1]:
!pip install numpy
!pip insatll tensorflow
!pip install opencv-python
!pip install matplotlib

ERROR: unknown command "insatll" - maybe you meant "install"


# Importing Modules

In [2]:
import numpy
import matplotlib.pyplot as plt
import tensorflow as tf
import cv2

In [5]:
def normalize(X_train,X_test):
  X_train = tf.keras.utils.normalize(X_train, axis=1)
  X_test = tf.keras.utils.normalize(X_test, axis=1)
  return X_train, X_test

In [4]:
def build_model():
  model = tf.keras.models.Sequential() #Selecting model
  model.add(tf.keras.layers.Flatten()) #FLattening inputs
  model.add(tf.keras.layers.Dense(units=128, activation=tf.nn.relu)) #Input layer
  model.add(tf.keras.layers.Dense(units=128, activation=tf.nn.relu)) #Hidden Dense Layer
  model.add(tf.keras.layers.Dense(units=10, activation=tf.nn.softmax)) #Output Dense Layer

  return model

In [3]:
def train_model():
  mnist = tf.keras.datasets.mnist
  (X_train, y_train), (X_test, y_test) = mnist.load_data()

  X_train, X_test = normalize(X_train,X_test)

  model = build_model()

  # Compiling and optimizing model
  model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

  # Training the model
  model.fit(X_train, y_train, epochs=3)

  # Evaluating the model
  val_loss, val_acc = model.evaluate(X_test, y_test)
  print(val_loss)
  print(val_acc)

  # Saving the model
  model.save('digit_recognizer.model')

In [6]:
train_model()

11490434/11490434 [==============================] - 0s 0us/step
Epoch 1/3
1875/1875 [==============================] - 9s 4ms/step - loss: 0.2659 - accuracy: 0.9209
Epoch 2/3
1875/1875 [==============================] - 8s 4ms/step - loss: 0.1087 - accuracy: 0.9666
Epoch 3/3
313/313 [==============================] - 1s 2ms/step - loss: 0.1011 - accuracy: 0.9692
0.10106157511472702
0.9692000150680542


In [8]:
# Load the model
model = tf.keras.models.load_model('digit_recognizer.model')

In [9]:
model

In [35]:
def load_custom_image():
  try:
    image_name = "Six.png"
    img = cv2.imread(image_name)
    img = numpy.invert(numpy.array([img]).astype(bool))
    img = cv2.resize(img,(28,28))
    prediction = model.predict(img)
    print("The image is: {}".format(np.argmax(prediction)))
    plt.imshow(img[0], cmap = plt.cm.binary)
    plt.show()
  except:
    print("Error reading provided image!!")

In [36]:
load_custom_image()

Error reading provided image!!
